# Vehicle Capacity Routing Problem

## Group member

### Group name
Nong Tong and P Arm

### Group member
1. Natthawat				64070701603
2. Phaprararath 	 	64070701610



## Import Library

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from random import random, randint, choice, choices, shuffle, sample
import math
from math import ceil, floor
import copy
from copy import deepcopy
import time
import operator
import pandas as pd
import itertools
from decimal import *
getcontext().prec = 2
%matplotlib inline

## Define function

### `sum_distance`
Sum the distance of path
### Parameter:
d - Distance between node\
x - Traveled node or path
display - Display how distance travel

In [2]:
def sum_distance(d, x, display=False):
    distance = 0
    for i in range(len(x)-1):
        distance += d[x[i]][x[i+1]]
        if display == True:
            print("%d -> %d: %.02lf" %(x[i], x[i+1],d[x[i]][x[i+1]]))
            print("Sum Distance: ",distance)
            print("==========")

    distance += d[x[-1]][0]
    if display == True:
        print("%d -> %d: %.02lf" %(x[-1], x[0],d[x[-1]][x[0]]))
        print("Sum Distance: ",distance)
        print("==========")
    return distance

In [3]:
# Example
d_example = [[0,4,5],
             [4,0,3],
             [5,3,0]]
x_example = [0,1,2]

distance = sum_distance(d_example, x_example)
print("Distance: %.02lf" % distance)

Distance: 12.00


In [4]:
# Example
d_example = [[0,4,5],
             [4,0,3],
             [5,3,0]]
x_example = [0,1,2]

distance = sum_distance(d_example, x_example, True)
print("Distance: %.02lf" % distance)

0 -> 1: 4.00
Sum Distance:  4
1 -> 2: 3.00
Sum Distance:  7
2 -> 0: 5.00
Sum Distance:  12
Distance: 12.00


### `sum_demand`
Sum every demand of each customer in path
#### Parameter:
w - Demand of customer in each city\
x - Traveled node or path. Can be empty to calculate all demand that need to serve\
display - Display the constraint in each travel_node

In [5]:
def sum_demand(w, x =[], display=False):
    demand = 0
    if x != []:
        for x_item in x:
            demand += w[x_item]
            if display:
                print("Node:", x_item)
                print("Demand: %.02lf" % w[x_item])
                print("Sum Demand: %.02lf" % demand)
                print("==========")
    else:
        for w_item in w:
            demand += w_item
    return demand

In [6]:
# Example
w_example = [0, 0.1, 0.2, 0.4]
x_example = [0,2]

demand = sum_demand(w_example, x_example)
print("Demand: %.02lf" % demand)

demand = sum_demand(w_example)
print("Demand: %.02lf" % demand)

Demand: 0.20
Demand: 0.70


In [7]:
# Example
# w_example = [0, 0.1, 0.2]
w_example = [0, 0.1, 0.2, 0.4]
x_example = [0,3,1]

demand = sum_demand(w_example, x_example, True)
print("Demand: %.02lf" % demand)

Node: 0
Demand: 0.00
Sum Demand: 0.00
Node: 3
Demand: 0.40
Sum Demand: 0.40
Node: 1
Demand: 0.10
Sum Demand: 0.50
Demand: 0.50


### `ordered_crossover_path`
Ordered crossover of 2 paths. Concate all path and then crossover on position.\
If don't specific position to cut, it will random position to crossover.\
When crossover, it will not consider node = 0.
#### Parameter:
x1 - First parent (path)\
x2 - Second parent (path\
cut_pos1 - first position to cut. (Don't count the node 0)\
cut_pos2 - second position to cut. (Don't count the node 0)

In [8]:
def ordered_crossover_path(x1, x2, cut_pos1 = None, cut_pos2 = None):
    p1 = []
    p2 = []
    for i in range(len(x1)):
        p1 = p1 + x1[i][1:]
    for i in range(len(x2)):
        p2 = p2 + x2[i][1:]

    max_len = len(p1) if len(p1) > len(p2) else len(p2)
    
    #Random pos to crosover
    cut_pos1 = cut_pos1 if cut_pos1 else randint(1, floor(max_len*0.5))
    cut_pos2 = cut_pos2 if cut_pos2 else randint(cut_pos1+1, max_len)
    
    # cut_node
    cut_node = p1[cut_pos1:cut_pos2]
    
    # Filter p2 that don't have in cut_node
    filtered_p2 = list(filter(lambda p2_i: p2_i not in cut_node, p2))
    
    # Assign the rest of child to be p2
    prefix = filtered_p2[0:cut_pos1]
    suffix = filtered_p2[cut_pos1:]
    child = prefix+cut_node+suffix
    
    child.insert(0,0)
    return child

In [9]:
x1_example = [[0,3,2,7,6,5,1,4]]
x2_example = [[0,1,3,2,5,4]]
x_result = ordered_crossover_path(x1_example, x2_example)
print(x_result)

[0, 1, 3, 7, 6, 5, 2, 4]


In [10]:
x1_example = [[0,1,3],[0, 2,5,4]] 
x2_example = [[0,3,4,2],[0,7,6,5,1]]
x_result = ordered_crossover_path(x1_example, x2_example, cut_pos1 = 2, cut_pos2 = 5)
print(x_result)

[0, 3, 7, 2, 5, 4, 6, 1]


### `mutate_path`
Shuffle node inside each path
#### Parameter:
parent - Solution of path

In [11]:
x = [1,2,3]
sample(x,k=len(x))
x

[1, 2, 3]

In [12]:
def mutate_path(parent):
    parent = deepcopy(parent)

    # Random path
    path_pos = randint(0,len(parent)-1)
    pos1, pos2 = choices(range(1, len(parent[path_pos])), k=2)
    count = 0
    while pos1 == pos2:
        path_pos = randint(0,len(parent)-1)
        pos1, pos2 = choices(range(1, len(parent[path_pos])), k=2)
        if ++count == 5:
            break
    temp = parent[path_pos][pos1]
    parent[path_pos][pos1] = parent[path_pos][pos2]
    parent[path_pos][pos2] = temp
    return parent

In [13]:
x_example = [[0, 6, 5, 3, 8],[0, 1, 2, 4, 7]]
print(mutate_path(x_example))

[[0, 8, 5, 3, 6], [0, 1, 2, 4, 7]]


### `min_n_node_from_demand`
Find the number of minimum node to travel to serve all customer demand constraint
#### Parameter:
w - Demand of customer in each city

In [14]:
def minimum_n_node_from_demand(w):
    count_node = 0
    for w_item in w:
        if w_item > 0:
            count_node += 1
    return count_node

In [15]:
w_example = [0.1, 0.2, 0.4, 0, 2, 0]

minimum_node = minimum_n_node_from_demand(w_example)
print(minimum_node)

4


### `find_unserved_node`
Find node that haven't serve the demand
#### Parameter:
w - Demand of customer in each city\
x - Node that traveled

In [16]:
def find_unserved_node(w, x):
    if w == []:
        return []
    w = w.copy()
    for x_item in x:
        w[x_item] = 0
    unserved_node =[]
    for i in range(len(w)):
        if w[i] != 0:
            unserved_node.append(i)
    return unserved_node            

In [17]:
w_example = [0, 1, 0, 2, 0.5, 0, 2, 0]
x_example = [0, 1, 2]
x_result = find_unserved_node(w_example, x_example)
print(x_result)

[3, 4, 6]


In [18]:
w_example = [0, 1, 0, 2, 0.5, 0, 2, 0]
x_example = []
x_result = find_unserved_node(w_example, x_example)
x_result

[1, 3, 4, 6]

### `split_path_from_demand`
Split the path from demand
#### Parameter:
x - path\
w - Demand constraint\
c - Capacity

In [19]:
def split_path_from_demand(x, w, c):
    paths = []
    path = []
    demand = 0
    if w == []:
        return [x]
    for index, x_item in enumerate(x):
        if demand + w[index] > c:
            paths.append(path)
            path = [0]
            demand = 0  
        demand += w[index]
        path.append(x_item)
    paths.append(path)
    return paths

In [20]:
x_example = [0, 3, 1, 2, 6, 7, 4, 5]

split_path_from_demand(x_example, [], 0)

[[0, 3, 1, 2, 6, 7, 4, 5]]

In [21]:
x_example = [0, 3, 1, 2, 6, 7, 4, 5]
# w_example = [0, 3, 2, 1, 3, 1, 1 ,2]
w_example = [0, 5, 2, 1, 3, 1, 1 ,2]
c_example = 5

split_path_from_demand(x_example, w_example, c_example)

[[0, 3], [0, 1, 2], [0, 6, 7, 4], [0, 5]]

### `generate_random_path`
Generate the random path at least possible minimum node to n node.\
Also, split the path depends on demand constraint.
#### Parameter:
N - Number of node\
w - Demand constraint\
c - Capacity\
n - number of minimum customer node (don't count node = 0)\
n_truck - number of truck

In [22]:
all_node = [i for i in range(1,9)]
all_node[:5]

[1, 2, 3, 4, 5]

In [23]:
def generate_random_path(N, w=[], c=999, n=-1, n_path=1):
    w = [0 for _ in range(N+1)] if w == [] else w # If w is empty list, set weight to 0  
    
    n = n if n != -1 else minimum_n_node_from_demand(w) # If not set n, set n to possible minimum distance

    # set all node
    all_node = [i for i in range(1,N)]
    shuffle(all_node)

    # Select only n node
    rand_path = all_node[:n]
    
    # Add node that doesn't serve demand and shuffle again
    rand_path = rand_path + find_unserved_node(w, rand_path)
    shuffle(rand_path)
    
    # Add initial node
    rand_path.insert(0,0)
    
    # If have weight constraint
    paths = split_path_from_demand(rand_path, w, c)
    

    # If the number of path more than number of truck, random agian
    if len(paths) > n_path:
        paths = generate_random_path(N, w, c, n, n_path)

    return paths

In [24]:
def generate_random_path(N, w=[], c=999, n=-1, n_path=1):
    if w == []:
        w = [0 for _ in range(N+1)]
        all_node = [i for i in range(1,N)]
    else:
        n = n if n != -1 else minimum_n_node_from_demand(w) # If not set n, set n to possible minimum distance
        # print(w)
        # print(f'n is {n}')
        all_node = []
        for index, data in enumerate(w):
            if data != 0:
                all_node.append(index)
        # print(all_node)
    shuffle(all_node)

    # Select only n node
    rand_path = all_node[:n]
    
    # Add initial node
    rand_path.insert(0,0)
    
    # If have weight constraint
    paths = split_path_from_demand(rand_path, w, c)
    

    # If the number of path more than number of truck, random agian
    if len(paths) > n_path:
        paths = generate_random_path(N, w, c, n, n_path)

    return paths

In [25]:
path_result = generate_random_path(N=15, n=10)
print("Path:", path_result)

Path: [[0, 9, 4, 10, 14, 12, 1, 8, 7, 2, 3]]


In [26]:
#1,2,3,4,5,6,8,10
w_example = [0, 0.5, 1, 0.5, 1.5, 2, 2, 0, 0.5, 0, 0.1]
c_example = 5
paths_result = generate_random_path(N=10, w=w_example, c=c_example, n_path = 2)
print("Paths:", paths_result)

Paths: [[0, 2, 5, 3, 6], [0, 4, 8, 10, 1]]


### `sort_path_distance`
Sort every path from the distance
#### Parameter:
paths - All paths\
distances - All distance

In [27]:
def sort_path_distance(paths, distances, reverse = False):
    # Pair distances and path
    zipped_lists = zip(distances, paths)
    
    # Sort by first element of each pair
    sorted_zipped_lists = sorted(zipped_lists, reverse=reverse)
    
    sorted_paths = [path for _, path in sorted_zipped_lists]
    sorted_distance = [distance for distance, _ in sorted_zipped_lists]
    return sorted_paths, sorted_distance

In [28]:
x_example = [[[0,2,1,4],[0,3,5,6]],
             [[0,3,2,1],[0,4,5,6]],
             [[0,1,4,5],[0,1,3,6]]
            ]
d_example = [1,2,0]

print("Before sorted: ")
print(x_example)
print(d_example)
x_result, d_result = sort_path_distance(paths = x_example, distances = d_example)
print("After sorted")
print(x_result)
print(d_result)

Before sorted: 
[[[0, 2, 1, 4], [0, 3, 5, 6]], [[0, 3, 2, 1], [0, 4, 5, 6]], [[0, 1, 4, 5], [0, 1, 3, 6]]]
[1, 2, 0]
After sorted
[[[0, 1, 4, 5], [0, 1, 3, 6]], [[0, 2, 1, 4], [0, 3, 5, 6]], [[0, 3, 2, 1], [0, 4, 5, 6]]]
[0, 1, 2]


In [29]:
def sort_path_distance2(paths, distances, diff, fitness, reverse = False):
    d = {'paths': paths, 'distances': distances,'diff': diff,'fitness': fitness}
    df = pd.DataFrame(data=d)
    df = df.sort_values(by=['fitness'],ascending=False)
    sorted_paths = df['paths'].tolist()
    sorted_distance = df['distances'].tolist()
    sorted_diff = df['diff'].tolist()
    sorted_fitness = df['fitness'].tolist()
    return sorted_paths, sorted_distance, sorted_diff, sorted_fitness

In [30]:
x_example = [[[0,2,1,4],[0,3,5,6]],
             [[0,3,2,1],[0,4,5,6]],
             [[0,1,4,5],[0,1,3,6]]
            ]
d_example = [0,2,1]
diff = [5,7,3]
fitness = [13,9,11]

print("Before sorted: ")
print(x_example)
print(d_example)
x_result, d_result, diff_result, fitness_result = sort_path_distance2(paths = x_example, distances = d_example, diff = diff,fitness =fitness )
print("After sorted")
print(x_result)
print(d_result)
print(diff_result)
print(fitness_result)

Before sorted: 
[[[0, 2, 1, 4], [0, 3, 5, 6]], [[0, 3, 2, 1], [0, 4, 5, 6]], [[0, 1, 4, 5], [0, 1, 3, 6]]]
[0, 2, 1]
After sorted
[[[0, 2, 1, 4], [0, 3, 5, 6]], [[0, 1, 4, 5], [0, 1, 3, 6]], [[0, 3, 2, 1], [0, 4, 5, 6]]]
[0, 1, 2]
[5, 3, 7]
[13, 11, 9]


### `unique_path`
Remove duplicate path.
#### Paremeter:
x - All paths

In [31]:
def unique_path(x): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for sol in x:
        # check if exists in unique_list or not 
        if sol not in unique_list: 
            unique_list.append(deepcopy(sol)) 
    return unique_list

In [32]:
x_example = [[[0, 1, 3, 2, 4]], [[0,2, 1, 3, 4]], [[0, 2, 4, 5, 1]], [[0, 2, 4, 5, 1]]]
unique_path(x_example)

[[[0, 1, 3, 2, 4]], [[0, 2, 1, 3, 4]], [[0, 2, 4, 5, 1]]]

### `roulette_wheel_random`
Use roulette wheel to random.
#### Paremeter:
y - weight to random\
x - item to random\
n - number of item to random\
min_bool - Decided min and max weight (default: max)

In [33]:
def roulette_wheel_random(x, y, n=1, min_bool=False):
    y=np.array(y)
    prob = (np.sum(y)-y)/(np.sum(np.sum(y)-y)) if min_bool else y/np.sum(y)
    # print(f"""len y is {len(y)} n is {n}, prob = {prob}""")
    pos = np.random.choice(len(y), n, replace=False, p = prob)
    rand_item = []
    for pos_item in pos:
        rand_item.append(x[pos_item])
    return rand_item

In [34]:
x_example = [[[0,1,2,3],[0,4,5,6]],
             [[0,1,3,2],[0,5,4,6]],
             [[0,2,1,3],[0,4,6,5]],
             [[0,3,4,5],[0,1,2,6]]
            ]
y_example = [10,30,20,5]
roulette_wheel_random(x_example, y_example)

[[[0, 1, 3, 2], [0, 5, 4, 6]]]

In [35]:
roulette_wheel_random(x_example, y_example, n=2, min_bool = True)

[[[0, 2, 1, 3], [0, 4, 6, 5]], [[0, 3, 4, 5], [0, 1, 2, 6]]]

## Algorithm

### `brute_force`
#### Parameter:
d - Distance\
n_node - Number of node\
w - Demand of each customer\
n_path - No. of possible path (trucks)\
c - Capacity\
n - number of customer node (Use when don't have any demand) (Don't count node = 0)\
path - Current path to calculate

In [36]:
def brute_force(d, n_node, w=[], n_path=1, c=999, n=-1, path=[0]):
    min_d = 9999
    min_x = []
    if ((n != -1) and (len(path) == n+1)):
        distance = sum_distance(d=d, x=path)
        return distance, path
    
    x = split_path_from_demand(path, w, c)
    # Impossible solution
    if(len(x) > n_path):
        return 9999, []
    
    unserved_node = find_unserved_node(w, path)
    distance = 0
    if (n == -1 and unserved_node == []):
        for x_item in x:
            distance += sum_distance(d=d, x=x_item)
        return distance, x

    node_all = [i for i in range(0,n_node)]
    node_select = [item for item in node_all if item not in path]
    for i in node_select:
        new_path = path + [i]
        distance, x = brute_force(d=d,n_node=n_node, w=w, n_path=n_path, c=c, n=n, path=new_path)
        if distance < min_d:
            min_d = distance
            min_x = x
    return min_d, min_x

In [37]:
# def brute_force2(d, n_node, w=[], n_path=1, c=999, n=-1, path=[0]):
#     min_d = 9999
#     min_diff = 9999
#     min_x = []
#     diff = 0
#     if (n != -1 and len(path) == n+1):
#         distance = sum_distance(d=d, x=path)
#         # print(f'''{distance} {path}''')
#         return distance, path
    
#     x = split_path_from_demand(path, w, c)
#     # print(f'''split_path_from_demand {x}''')
#     # Impossible solution
#     if(len(x) > n_path):
#         return 9999, []
    
#     unserved_node = find_unserved_node(w, path)
#     distance = 0
#     if (n == -1 and unserved_node == []):
#         for x_item in x:
#             distance += sum_distance(d=d, x=x_item)
#             print(f'''{distance} {x}''')
#         return distance, x

#     node_all = [i for i in range(0,n_node)]
#     node_select = [item for item in node_all if item not in path]
    
#     for i in node_select:
#         new_path = path + [i]
#         distance, x = brute_force(d=d,n_node=n_node, w=w, n_path=n_path, c=c, n=n, path=new_path)
#         for index, data in enumerate(x):
#             # print(f"""Distance of truck {data} {sum_distance(d=d,x = data)}""")
#             diff = abs(sum_distance(d=d,x = data) - diff)
#         if (distance < min_d) and (diff < min_diff):
#             min_d = distance
#             min_x = x
#             min_diff = diff
#     return min_d, min_x, min_diff

In [38]:
def brute_force2(d, n_node, w=[], n_path=1, c=999, n=-1, path=[0], min_dis_factor=1,min_dif_factor=1):
    min_d = 9999
    min_diff = 9999
    min_x = []
    all_node = []
    permu_list = []
    splited_path = []
    total_distance = []
    total_diff = []
    total_fit = []
    print(w)
    if w:
        for index, data in enumerate(w):
            if data != 0:
                all_node.append(index)
    # print(all_node)
    else:
        all_node = range(1,len(d))
        
    permu_turple = list(itertools.permutations(all_node))
    for permu in permu_turple:
        permu = list(permu)
        permu.insert(0,0)
        permu_list.append(permu)
        splited = split_path_from_demand(permu, w, c)
        splited_path.append(splited)
        mysum = 0
        tempSum = 0
        diff = 0
        for each_path in splited:
            tempSum = sum_distance(d=d, x=each_path)
            mysum = mysum + tempSum
            diff = abs(tempSum - diff)
        total_fit.append(((1/(diff+0.1))*min_dif_factor)*((1/(distance+0.1))*min_dis_factor))
        total_diff.append(diff)
        total_distance.append(mysum)
    
    # chunks = [data[x:x+100] for x in range(0, len(data), 100)]
    # print(total_diff)
    fine_path, fine_distance, fine_diff, fine_fitness = sort_path_distance2(paths = splited_path, distances = total_distance, diff = total_diff, fitness = total_fit, reverse = True)
    return fine_distance, fine_path, fine_diff

In [39]:
def brute_force3(d, n_node, w=[], n_path=1, c=999, n=-1, path=[0]):
    min_d = 9999
    min_diff = 9999
    min_x = []
    all_node = []
    permu_list = []
    splited_path = []
    total_distance = []
    total_diff = []
    total_fit = []
    print(w)
    for index, data in enumerate(w):
        if data != 0:
            all_node.append(index)
    print(all_node)
    permu_turple = list(itertools.permutations(all_node))
    
    for permu in permu_turple:
        permu = list(permu)
        permu.insert(0,0)
        permu_list.append(permu)
        splited = split_path_from_demand(permu, w, c)
        splited_path.append(splited)
        mysum = 0
        tempSum = 0
        diff = 0
        for each_path in splited:
            tempSum = sum_distance(d=d, x=each_path)
            mysum = mysum + tempSum
            diff = abs(tempSum - diff)
        total_fit.append((1/(diff+0.1)) * (1/(mysum+0.1)))
        total_diff.append(diff)
        total_distance.append(mysum)
    fine_path, fine_distance, fine_diff, fine_fitness = sort_path_distance2(paths = splited_path, distances = total_distance, diff = total_diff, fitness = total_fit, reverse = True)
    return fine_distance, fine_path, fine_diff

In [40]:
def brute_force4(d, n_node, w=[], n_path=1, c=999, n=-1, path=[0]):
    min_d = 9999
    min_diff = 9999
    min_x = []
    all_node = []
    permu_list = []
    splited_path = []
    total_distance = []
    total_diff = []
    total_fit = []
    print(w)
    if w:
        for index, data in enumerate(w):
            if data != 0:
                all_node.append(index)
    # print(all_node)
    else:
        all_node = range(1,len(d))
    
    for permu in permu_turple:
        permu = list(permu)
        permu.insert(0,0)
        permu_list.append(permu)
        splited = split_path_from_demand(permu, w, c)
        splited_path.append(splited)
        mysum = 0
        tempSum = 0
        diff = 0
        for each_path in splited:
            tempSum = sum_distance(d=d, x=each_path)
            mysum = mysum + tempSum
            diff = abs(tempSum - diff)
        total_fit.append((1/(mysum+0.1)))
        total_diff.append(diff)
        total_distance.append(mysum)
    fine_path, fine_distance, fine_diff, fine_fitness = sort_path_distance2(paths = splited_path, distances = total_distance, diff = total_diff, fitness = total_fit, reverse = True)
    return fine_distance, fine_path, fine_diff

### `genetic_algorithm`
#### Parameter:
d - Distance\
n_node - Number of node\
w - Demand of each customer\
n_path - No. of possible path (trucks)\
c - Capacity\
n - number of customer node (Use when don't have any demand) (Don't count node = 0)\
n_pop - Population\
generation - Generation\
cr - Crossover rate\
mr - Mutation rate\
rr - Reproduction rate\
sp - Survival percentage (Percentage of good solutions to be next gen)

In [41]:
# Crossover
def crossover(n_node, parents, obj_parents, n, w, c, n_crossover, n_path):
    x_crossover = []
    while len(x_crossover) < n_crossover:
        parent1, parent2 = roulette_wheel_random(parents, obj_parents, n=2, min_bool=True)
        child = ordered_crossover_path(parent1, parent2)
        child = split_path_from_demand(child, w, c)
        # If the number of path more than number of truck, random
        while (len(child) > n_path) or (child in x_crossover):
            parent1, parent2 = roulette_wheel_random(parents, obj_parents, n=2, min_bool=True)
            child = ordered_crossover_path(parent1, parent2)
            child = split_path_from_demand(child, w, c)
        x_crossover.append(child)
    return x_crossover

def mutate(parents, mr):
    x_mutate = deepcopy(parents)
    for i in range(len(x_mutate)):
        if random() < mr:
            x_mutate[i] = mutate_path(x_mutate[i])
    return x_mutate

In [42]:
def genetic_algorithm(d, n_node, w=[], n_path=1, c=999, n=-1, n_pop=100,\
                      generation=300, cr=0.7, mr=0.1):
    bests_path = []
    bests_distance = []
    best_id = 0

    # Initial population
    x = []
    
    for _ in range(n_pop):
        sol = generate_random_path(N=n_node, w=w, c=c, n=n, n_path = n_path)
        x.append(sol)
    
    for _ in range(generation):
        x = unique_path(x)
        
        y = []
        # Evaluate
        for x_item in x:
            distance = 0
            for path in x_item:
                distance += sum_distance(d=d, x=path)
            y.append(distance)
        
        x, y = sort_path_distance(paths = x, distances = y)        
        
        # Get bests value
        if best_id == 0 or y[0] < bests_distance[best_id-1]:
            bests_path.append(x[0])
            bests_distance.append(y[0])
            best_id+=1
            
        # Survival
        n_crossover = ceil(n_pop * cr)
        n_survival = n_pop - n_crossover
        x_survival = deepcopy(x[:n_survival])
        y_survival = deepcopy(y[:n_survival])
        
        # Crossover
        x_crossover = crossover(n_node, x, y, n, w, c, n_crossover, n_path)
        
        # Mutation
        x_mutate = mutate(x_crossover, mr)
        
        x = x_survival + x_mutate
    return bests_path, bests_distance    

In [43]:
def genetic_algorithm2(d, n_node, w=[], n_path=1, c=999, n=-1, n_pop=100,\
                      generation=300, cr=0.7, mr=0.1, min_dis_factor=1,min_dif_factor=1):
    bests_path = []
    bests_distance = []
    best_id = 0

    # Initial population
    x = []
    print("Initial population")
    for _ in range(n_pop):
        sol = generate_random_path(N=n_node, w=w, c=c, n=n, n_path = n_path)
        x.append(sol)
    # print(x)
    print("Finish population")
    
    print("Start generation")
    for iter_gen in range(generation):
        # print(f'''iter_gen is {iter_gen}''')
        x = unique_path(x)
        y = []
        z = []
        fitness = []
        # Evaluate
        for index, x_item in enumerate(x):
            # print(f"""{index} pop""")
            distance = 0
            diff = 0
            for path in x_item:
                # print(f""" {path} """)
                distance += sum_distance(d=d, x=path)
                diff = abs(sum_distance(d=d, x=path)-diff)
            if min_dif_factor == 0:
                fitness.append((1/(distance+0.1))*min_dis_factor)
            elif min_dis_factor == 0:
                fitness.append((1/(diff+0.1))*min_dif_factor)
            else:
                fitness.append(((1/(diff+0.1))*min_dif_factor)*((1/(distance+0.1))*min_dis_factor))
            z.append(diff)
            y.append(distance)
            # หา diff ของ path -> ทำ 1/diff * 1/distance = fitness -> หา max fitness
        
        # print("sort_path_distance2")
        x, y, z, fine_fitness = sort_path_distance2(paths = x, distances = y, diff = z, fitness = fitness, reverse = True)        
        
        # Get bests value
        if (best_id == 0) or (fine_fitness[0] > bests_distance[best_id-1]):
            bests_path.append(x[0])
            bests_distance.append(y[0])
            best_id+=1
            
        # Survival
        n_crossover = ceil(n_pop * cr)
        n_survival = n_pop - n_crossover
        x_survival = deepcopy(x[:n_survival])
        y_survival = deepcopy(y[:n_survival])
        
        # Crossover
        x_crossover = crossover(n_node, x, y, n, w, c, n_crossover, n_path)
        
        # Mutation
        x_mutate = mutate(x_crossover, mr)
        
        x = x_survival + x_mutate
    print("Finish generation")
    return bests_path, bests_distance    

## Define Input

Define number of node

In [44]:
# For small data set
n_node_small = 10 # Number of node
n_small = 7       # Number of minimum customer (Used for no constraint)

# For big data set
n_node_big = 15 # Number of node
n_big = 10      # Number of minimum customer (Used for no constraint)

Define number of capacity and truck

In [45]:
c_small = 4
c_big = 5
n_path = 2

Distance between each customer

In [46]:
# Distance between each customer
d_8_city = [[0, 10, 28, 23, 65, 16, 16, 42, 16],
            [10, 0, 76, 72, 150, 52, 52, 104, 42],
            [28, 76, 0, 106, 74, 48, 83, 28, 88],
            [23, 72, 106, 0, 180, 82, 82, 134, 18],
            [65, 150, 74, 180, 0, 122, 157, 56, 162],
            [16, 52, 48, 82, 122, 0, 59, 76, 64],
            [16, 52, 83, 82, 157, 59, 0, 111, 64],
            [42, 104, 28, 134, 56, 76, 111, 0, 116],
            [16, 42, 88, 18, 162, 64, 64, 116, 0]]

d_9_city = [[0, 10, 28, 23, 65, 16, 16, 42, 16, 21],
            [10, 0, 76, 72, 150, 52, 52, 104, 42, 62],
            [28, 76, 0, 106, 74, 48, 83, 28, 88, 58],
            [23, 72, 106, 0, 180, 82, 82, 134, 18, 92],
            [65, 150, 74, 180, 0, 122, 157, 56, 162, 132],
            [16, 52, 48, 82, 122, 0, 59, 76, 64, 34],
            [16, 52, 83, 82, 157, 59, 0, 111, 64, 69],
            [42, 104, 28, 134, 56, 76, 111, 0, 116, 106],
            [16, 42, 88, 18, 162, 64, 64, 116, 0, 74],
            [21, 62, 58, 92, 132, 34, 69, 106, 74, 0]]
            
d_10_city = [[0, 10, 28, 23, 65, 16, 16, 42, 16, 21, 85],
         [10, 0, 76, 72, 150, 52, 52, 104, 42, 62, 190],
         [28, 76, 0, 106, 74, 48, 83, 28, 88, 58, 186],
         [23, 72, 106, 0, 180, 82, 82, 134, 18, 92, 220],
         [65, 150, 74, 180, 0, 122, 157, 56, 162, 132, 172],
         [16, 52, 48, 82, 122, 0, 59, 76, 64, 34, 162],
         [16, 52, 83, 82, 157, 59, 0, 111, 64, 69, 197],
         [42, 104, 28, 134, 56, 76, 111, 0, 116, 106, 126],
         [16, 42, 88, 18, 162, 64, 64, 116, 0, 74, 202],
         [21, 62, 58, 92, 132, 34, 69, 106, 74, 0, 172],
         [85, 190, 186, 220, 172, 162, 197, 126, 202, 172, 0]]

d_11_city = [[0, 10, 28, 23, 65, 16, 16, 42, 16, 21, 85, 23],
         [10, 0, 76, 72, 150, 52, 52, 104, 42, 62, 190, 60],
         [28, 76, 0, 106, 74, 48, 83, 28, 88, 58, 186, 10],
         [23, 72, 106, 0, 180, 82, 82, 134, 18, 92, 220, 18],
         [65, 150, 74, 180, 0, 122, 157, 56, 162, 132, 172, 180],
         [16, 52, 48, 82, 122, 0, 59, 76, 64, 34, 162, 82],
         [16, 52, 83, 82, 157, 59, 0, 111, 64, 69, 197, 82],
         [42, 104, 28, 134, 56, 76, 111, 0, 116, 106, 126, 134],
         [16, 42, 88, 18, 162, 64, 64, 116, 0, 74, 202, 18],
         [21, 62, 58, 92, 132, 34, 69, 106, 74, 0, 172, 92],
         [85, 190, 186, 220, 172, 162, 197, 126, 202, 172, 0, 220],
         [23, 60, 106, 18, 180, 82, 82, 134, 18, 92, 220, 0]]

d_12_city = [[0, 10, 28, 23, 65, 16, 16, 42, 16, 21, 85, 23, 35],
         [10, 0, 76, 72, 150, 52, 52, 104, 42, 62, 190, 60, 90],
         [28, 76, 0, 106, 74, 48, 83, 28, 88, 58, 186, 106, 86],
         [23, 72, 106, 0, 180, 82, 82, 134, 18, 92, 220, 18, 120],
         [65, 150, 74, 180, 0, 122, 157, 56, 162, 132, 172, 180, 120],
         [16, 52, 48, 82, 122, 0, 59, 76, 64, 34, 162, 82, 62],
         [16, 52, 83, 82, 157, 59, 0, 111, 64, 69, 197, 82, 97],
         [42, 104, 28, 134, 56, 76, 111, 0, 116, 106, 126, 134, 114],
         [16, 42, 88, 18, 162, 64, 64, 116, 0, 74, 202, 18, 102],
         [21, 62, 58, 92, 132, 34, 69, 106, 74, 0, 172, 92, 72],
         [85, 190, 186, 220, 172, 162, 197, 126, 202, 172, 0, 220, 110],
         [23, 60, 106, 18, 180, 82, 82, 134, 18, 92, 220, 0, 120],
         [35, 90, 86, 120, 120, 62, 97, 114, 102, 72, 110, 120, 0]]

d_13_city = [[0, 10, 28, 23, 65, 16, 16, 42, 16, 21, 85, 23, 35, 70],
         [10, 0, 76, 72, 150, 52, 52, 104, 42, 62, 190, 60, 90, 110],
         [28, 76, 0, 106, 74, 48, 83, 28, 88, 58, 186, 106, 86, 146],
         [23, 72, 106, 0, 180, 82, 82, 134, 18, 92, 220, 18, 120, 110],
         [65, 150, 74, 180, 0, 122, 157, 56, 162, 132, 172, 180, 120, 215],
         [16, 52, 48, 82, 122, 0, 59, 76, 64, 34, 162, 82, 62, 117],
         [16, 52, 83, 82, 157, 59, 0, 111, 64, 69, 197, 82, 97, 74],
         [42, 104, 28, 134, 56, 76, 111, 0, 116, 106, 126, 134, 114, 283],
         [16, 42, 88, 18, 162, 64, 64, 116, 0, 74, 202, 18, 102, 112],
         [21, 62, 58, 92, 132, 34, 69, 106, 74, 0, 172, 92, 72, 127],
         [85, 190, 186, 220, 172, 162, 197, 126, 202, 172, 0, 220, 110, 255],
         [23, 60, 106, 18, 180, 82, 82, 134, 18, 92, 220, 0, 120, 130],
         [35, 90, 86, 120, 120, 62, 97, 114, 102, 72, 110, 120, 0, 155],
         [70, 110, 146, 110, 215, 117, 74, 283, 112, 127, 255, 130, 155, 0]]

d_14_city = [[0, 10, 28, 23, 65, 16, 16, 42, 16, 21, 85, 23, 35, 70, 23],
         [10, 0, 76, 72, 150, 52, 52, 104, 42, 62, 190, 60, 90, 110, 65],
         [28, 76, 0, 106, 74, 48, 83, 28, 88, 58, 186, 106, 86, 146, 106],
         [23, 72, 106, 0, 180, 82, 82, 134, 18, 92, 220, 18, 120, 110, 10],
         [65, 150, 74, 180, 0, 122, 157, 56, 162, 132, 172, 180, 120, 215, 180],
         [16, 52, 48, 82, 122, 0, 59, 76, 64, 34, 162, 82, 62, 117, 82],
         [16, 52, 83, 82, 157, 59, 0, 111, 64, 69, 197, 82, 97, 74, 82],
         [42, 104, 28, 134, 56, 76, 111, 0, 116, 106, 126, 134, 114, 283, 134],
         [16, 42, 88, 18, 162, 64, 64, 116, 0, 74, 202, 18, 102, 112, 18],
         [21, 62, 58, 92, 132, 34, 69, 106, 74, 0, 172, 92, 72, 127, 92],
         [85, 190, 186, 220, 172, 162, 197, 126, 202, 172, 0, 220, 110, 255, 220],
         [23, 60, 106, 18, 180, 82, 82, 134, 18, 92, 220, 0, 120, 130, 10],
         [35, 90, 86, 120, 120, 62, 97, 114, 102, 72, 110, 120, 0, 155, 120],
         [70, 110, 146, 110, 215, 117, 74, 283, 112, 127, 255, 130, 155, 0, 120],
         [23, 65, 106, 10, 180, 82, 82, 134, 18, 92, 220, 10, 120, 120, 0]]

Demand of each customer

In [47]:
w_small = [0, 0.3, 1.8, 0.5, 0.9, 1.3, 1.5, 0.3, 0.4, 0]

w1_8  = [0] + [0, 0.8, 0, 0.9, 1.3, 1.5, 0, 0]
w1_9  = [0] + [0, 0.8, 0, 0.9, 1.3, 1.5, 0, 0, 0.3]
w1_10 = [0] + [0, 0.8, 0, 0.9, 1.3, 1.5, 0, 0, 0.3, 0.2]
w1_11 = [0] + [0, 0.8, 0, 0.9, 1.3, 1.5, 0, 0, 0.3, 0.2, 1]
w1_12 = [0] + [0, 0.8, 0, 0.9, 1.3, 1.5, 0, 0, 0.3, 0.2, 1, 0]
w1_13 = [0] + [0, 0.8, 0, 0.9, 1.3, 1.5, 0, 0, 0.3, 0.2, 1, 0, 0.6]
w1_14 = [0] + [0, 0.8, 0, 0.9, 1.3, 1.5, 0, 0, 0.3, 0.2, 1, 0, 0.6, 0.4]

w2_8  = [0] + [0.2, 0.8, 0.3, 0.7, 0.5, 1, 1, 1]
w2_9  = [0] + [0.2, 0.8, 0.3, 0.7, 0.5, 1, 1, 1, 2]
w2_10 = [0] + [0.2, 0.8, 0.3, 0.7, 0.5, 1, 1, 1, 2, 0.1]
w2_11 = [0] + [0.2, 0.8, 0.3, 0.7, 0.5, 1, 1, 1, 2, 0.1, 0.1]
w2_12 = [0] + [0.2, 0.8, 0.3, 0.7, 0.5, 1, 1, 1, 2, 0.1, 0.1, 0.2]
w2_13 = [0] + [0.2, 0.8, 0.3, 0.7, 0.5, 1, 1, 1, 2, 0.1, 0.1, 0.2, 0.2]
w2_14 = [0] + [0.2, 0.8, 0.3, 0.7, 0.5, 1, 1, 1, 2, 0.1, 0.1, 0.2, 0.2, 0.3]

w3 = [0] + [2, 0.2, 0.5, 0.1, 1.3, 0.1, 1.5, 1.8, 0, 0, 0.2, 0.3, 0.4, 0.4]
w4 = [0] + [0.5, 1, 1.5, 0, 0.2, 0, 0.8, 0.2, 0, 1, 0, 0.1, 0.5, 1.2]

## Result

### Phase 1: Shortest path, no demand constraint

In [48]:
def ResultPhase1(d_small, n_node_small, n_small):
    for i in range(0,5):
        print("Round ",i+1)
        print("Genetic Algorithm") 
        stime = time.time()
        x, y = genetic_algorithm(d=d_small, n_node=n_node_small, n=n_small)
        print("Time used: %.0f seconds" % ( time.time() - stime))
        print("The shortest path is: ", x[-1])
        print("Distance: ", y[-1]) 
        print("----------------------------------------------") 
    print("Brute Force") 
    stime = time.time()
    x = brute_force(d=d_small, n_node=n_node_small, n=n_small)
    print("Time used: %.0f seconds" % ( time.time() - stime))
    print("The shortest path is: ", x[1])
    print("Distance: ", x[0]) 
    print("==============================================") 

In [49]:
def ResulฺtBruteForcePhase1_2(d, w, n_node_small, c=999,min_dis_factor=1,min_dif_factor=1):
    print("Brute Force")
    diff = 0
    stime = time.time()
    min_d, min_x, min_diff = brute_force2(d=d, w=w, n_node=n_node_small, c=c,min_dis_factor=min_dis_factor,min_dif_factor=min_dif_factor)
    print("Time used: %.0f seconds" % ( time.time() - stime))
    print("The shortest path is: ", min_x[0])
    print("Distance: ", min_d[0])
    for index, data in enumerate(min_x[0]):
        print(f"""Distance of truck {data} {sum_distance(d=d,x = data)}""")
        diff = abs(sum_distance(d=d,x = data) - diff)
    print(f"""Different distance of two truck {diff}""")
    print("==============================================") 

#### 8 City

In [50]:
ResultPhase1(d_8_city, n_node_small=9, n_small=8)

Round  1
Genetic Algorithm
Time used: 2 seconds
The shortest path is:  [[0, 3, 8, 1, 6, 5, 2, 7, 4]]
Distance:  391
----------------------------------------------
Round  2
Genetic Algorithm
Time used: 2 seconds
The shortest path is:  [[0, 3, 8, 1, 6, 5, 2, 7, 4]]
Distance:  391
----------------------------------------------
Round  3
Genetic Algorithm
Time used: 2 seconds
The shortest path is:  [[0, 3, 8, 1, 6, 5, 2, 7, 4]]
Distance:  391
----------------------------------------------
Round  4
Genetic Algorithm
Time used: 2 seconds
The shortest path is:  [[0, 4, 7, 2, 5, 6, 1, 8, 3]]
Distance:  391
----------------------------------------------
Round  5
Genetic Algorithm
Time used: 2 seconds
The shortest path is:  [[0, 4, 7, 2, 5, 6, 1, 8, 3]]
Distance:  391
----------------------------------------------
Brute Force
Time used: 0 seconds
The shortest path is:  [0, 3, 8, 1, 6, 5, 2, 7, 4]
Distance:  391


In [51]:
ResulฺtBruteForcePhase1_2(d_8_city, w=[], n_node_small=8,min_dis_factor=0.9,min_dif_factor=0.1)

Brute Force
[]
Time used: 0 seconds
The shortest path is:  [[0, 3, 8, 1, 6, 5, 2, 7, 4]]
Distance:  391
Distance of truck [0, 3, 8, 1, 6, 5, 2, 7, 4] 391
Different distance of two truck 391


#### 9 city

In [118]:
ResultPhase1(d_9_city, n_node_small=10, n_small=9)

Round  1
Genetic Algorithm
Time used: 2 seconds
The shortest path is:  [[0, 3, 8, 1, 6, 5, 9, 2, 7, 4]]
Distance:  435
----------------------------------------------
Round  2
Genetic Algorithm
Time used: 2 seconds
The shortest path is:  [[0, 4, 7, 2, 5, 9, 6, 1, 8, 3]]
Distance:  435
----------------------------------------------
Round  3
Genetic Algorithm
Time used: 2 seconds
The shortest path is:  [[0, 3, 8, 1, 6, 9, 5, 2, 7, 4]]
Distance:  435
----------------------------------------------
Round  4
Genetic Algorithm
Time used: 2 seconds
The shortest path is:  [[0, 4, 7, 2, 9, 5, 6, 1, 8, 3]]
Distance:  435
----------------------------------------------
Round  5
Genetic Algorithm
Time used: 2 seconds
The shortest path is:  [[0, 3, 8, 1, 6, 5, 9, 2, 7, 4]]
Distance:  435
----------------------------------------------
Brute Force
Time used: 2 seconds
The shortest path is:  [0, 3, 8, 1, 6, 5, 9, 2, 7, 4]
Distance:  435


In [117]:
ResulฺtBruteForcePhase1_2(d_9_city, w=[], n_node_small=8,min_dis_factor=0.9,min_dif_factor=0.1)

Brute Force
[]
Time used: 2 seconds
The shortest path is:  [[0, 4, 7, 2, 9, 5, 6, 1, 8, 3]]
Distance:  435
Distance of truck [0, 4, 7, 2, 9, 5, 6, 1, 8, 3] 435
Different distance of two truck 435


#### 10 city

In [120]:
ResultPhase1(d_10_city, n_node_small=11, n_small=10)

Round  1
Genetic Algorithm
Time used: 2 seconds
The shortest path is:  [[0, 10, 7, 4, 2, 9, 5, 6, 1, 8, 3]]
Distance:  627
----------------------------------------------
Round  2
Genetic Algorithm
Time used: 2 seconds
The shortest path is:  [[0, 10, 4, 7, 2, 9, 5, 6, 1, 8, 3]]
Distance:  627
----------------------------------------------
Round  3
Genetic Algorithm
Time used: 2 seconds
The shortest path is:  [[0, 3, 8, 1, 6, 9, 5, 2, 4, 7, 10]]
Distance:  627
----------------------------------------------
Round  4
Genetic Algorithm
Time used: 2 seconds
The shortest path is:  [[0, 3, 8, 1, 6, 5, 9, 2, 7, 4, 10]]
Distance:  627
----------------------------------------------
Round  5
Genetic Algorithm
Time used: 2 seconds
The shortest path is:  [[0, 10, 4, 7, 2, 9, 5, 6, 1, 8, 3]]
Distance:  627
----------------------------------------------
Brute Force
Time used: 16 seconds
The shortest path is:  [0, 3, 8, 1, 6, 5, 9, 2, 4, 7, 10]
Distance:  627


In [119]:
ResulฺtBruteForcePhase1_2(d_10_city, w=[], n_node_small=8,min_dis_factor=0.9,min_dif_factor=0.1)

Brute Force
[]
Time used: 16 seconds
The shortest path is:  [[0, 3, 8, 1, 6, 9, 5, 2, 4, 7, 10]]
Distance:  627
Distance of truck [0, 3, 8, 1, 6, 9, 5, 2, 4, 7, 10] 627
Different distance of two truck 627


#### 11 city

In [ ]:
#ResultPhase1(d_11_city, n_node_small=12, n_small=11)

In [ ]:
# ResulฺtBruteForcePhase1_2(d_11_city, w=[], n_node_small=8,min_dis_factor=0.9,min_dif_factor=0.1)

#### 12 city

In [ ]:
#ResultPhase1(d_12_city, n_node_small=13, n_small=12)

#### Big data set, no demand constraint

In [ ]:
# Big data set, no demand constraint
# x, y = genetic_algorithm(d=d_14_city, n_node=15, n=14)
# print("The shortest path is: ", x[-1])
# print("Distance: ", y[-1]) 

In [ ]:
# sum_distance(d_big, x[-1][0], True)

### Phase 2: Shortest path, with demand constraint

In [54]:
def ResultPhase2_1(d, n_node, w, n_path, c=999,cr=0.7,n_pop = 50,generation=100):
    print("Genetic Algorithm") 
    for i in range(0,5):
        print("Round ",i+1)
        diff = 0
        stime = time.time()
        x, y = genetic_algorithm(d=d, n_node=n_node, w=w, n_path=n_path, c=c, cr=cr,n_pop=n_pop,generation=generation)
        print("Time used: %.0f seconds" % ( time.time() - stime))
        print("The shortest path is: ", x[-1])
        print("Distance: ", y[-1])
        for index, data in enumerate(x[-1]):
            print(f"""Distance of truck {data} {sum_distance(d=d,x = data)}""")
            diff = abs(sum_distance(d=d,x = data) - diff)
        print(f"""Different distance of two truck {diff}""")
        print("----------------------------------------------") 

In [200]:
def ResultPhase2_2(d, n_node, w, n_path, c=999,cr=0.7,n_pop = 50,generation=100, min_dis_factor=1,min_dif_factor=1):
    avgDis=0
    avgDiff=0
    print("Genetic Algorithm") 
    for i in range(0,5):
        print("Round ",i+1)
        diff = 0
        stime = time.time()
        x, y = genetic_algorithm2(d=d, n_node=n_node, w=w, n_path=n_path, c=c, cr=cr,n_pop=n_pop,generation=generation, min_dis_factor=min_dis_factor,min_dif_factor=min_dif_factor)
        print("Time used: %.0f seconds" % ( time.time() - stime))
        print("The shortest path is: ", x[-1])
        print("Distance: ", y[-1])
        for index, data in enumerate(x[-1]):
            print(f"""Distance of truck {data} {sum_distance(d=d,x = data)}""")
            diff = abs(sum_distance(d=d,x = data) - diff)
        print(f"""Different distance of two truck {diff}""")
        avgDis += y[-1]
        avgDiff += diff
        print("----------------------------------------------")
    print(f'''avgDis is {avgDis/5}''')
    print(f'''avgDiff is {avgDiff/5}''')

In [176]:
def ResulฺtBruteForcePhase2_1(d, n_node, w, n_path, c=999):
    print("Brute Force")
    diff = 0
    stime = time.time()
    min_d, min_x = brute_force(d=d, n_node=n_node, w=w, n_path=n_path, c=c)
    print("Time used: %.0f seconds" % ( time.time() - stime))
    print("The shortest path is: ", min_x)
    print("Distance: ", min_d)
    for index, data in enumerate(min_x):
        print(f"""Distance of truck {data} {sum_distance(d=d,x = data)}""")
        diff = abs(sum_distance(d=d,x = data) - diff)
    print(f"""Different distance of two truck {diff}""")
    print("==============================================") 

In [177]:
def ResulฺtBruteForcePhase2_2(d, n_node, w, n_path, c=999):
    print("Brute Force") 
    stime = time.time()
    min_d, min_x, min_diff = brute_force2(d=d, n_node=n_node, w=w, n_path=n_path, c=c)
    print("Time used: %.0f seconds" % ( time.time() - stime))
    print("The shortest path is: ", min_x[0])
    print("Distance: ", min_d[0])
    print(f"""Different distance of two truck {min_diff[0]}""")
    print("==============================================") 

In [178]:
def ResulฺtBruteForcePhase2_3(d, n_node, w, n_path, c=999):
    print("Brute Force") 
    stime = time.time()
    min_d, min_x, min_diff = brute_force3(d=d, n_node=n_node, w=w, n_path=n_path, c=c)
    print("Time used: %.0f seconds" % ( time.time() - stime))
    print("The shortest path is: ", min_x[0])
    print("Distance: ", min_d[0])
    print(f"""Different distance of two truck {min_diff[0]}""")
    print("==============================================") 

In [ ]:
def ResulฺtBruteForcePhase2_4(d, n_node, w, n_path, c=999):
    print("Brute Force") 
    stime = time.time()
    min_d, min_x, min_diff = brute_force4(d=d, n_node=n_node, w=w, n_path=n_path, c=c)
    print("Time used: %.0f seconds" % ( time.time() - stime))
    print("The shortest path is: ", min_x[0])
    print("Distance: ", min_d[0])
    print(f"""Different distance of two truck {min_diff[0]}""")
    print("==============================================") 

#### 8 city

In [ ]:
ResultPhase2_1(d=d_8_city, n_node=9, w=w1_8, n_path=2, c=4,cr=0.3,n_pop=50,generation=100)

In [ ]:
ResultPhase2_2(d=d_8_city, n_node=9, w=w1_8, n_path=2, c=4, cr=0.3,n_pop=50,generation=100)

In [ ]:
ResultPhase2_2(d=d_8_city, n_node=9, w=w1_8, n_path=2, c=4,cr=0.4,n_pop=60,generation=150,min_dis_factor=1,min_dif_factor=1)

In [ ]:
# ResulฺtBruteForcePhase2_1(d=d_8_city, n_node=9, w=w1_8, n_path=2, c=4)

In [124]:
# ResulฺtBruteForcePhase2_2(d=d_8_city, n_node=9, w=w1_8, n_path=2, c=4)

Brute Force
[0, 0, 0.8, 0, 0.9, 1.3, 1.5, 0, 0]
Time used: 0 seconds
The shortest path is:  [[0, 4, 2, 5, 6]]
Distance:  262
Different distance of two truck 262


In [ ]:
ResulฺtBruteForcePhase2_3(d=d_8_city, n_node=9, w=w1_8, n_path=2, c=4)

In [ ]:
ResulฺtBruteForcePhase2_3(d=d_8_city, n_node=9, w=w2_8, n_path=2, c=4)

In [ ]:
ResulฺtBruteForcePhase2_4(d=d_8_city, n_node=9, w=w1_8, n_path=2, c=4)

In [ ]:
# ResultPhase2(d=d_8_city, n_node=9, w=w2_8, n_path=2, c=4)

In [ ]:
# ResulฺtBruteForcePhase2(d=d_8_city, n_node=9, w=w2_8, n_path=2, c=4)

#### 9 city

In [ ]:
ResultPhase2_1(d=d_9_city, n_node=10, w=w1_9, n_path=2,c=4,cr=0.4,n_pop=50,generation=100)

In [ ]:
ResultPhase2_2(d=d_9_city, n_node=10, w=w1_9, n_path=2,c=4,cr=0.4,n_pop=50,generation=100)

In [ ]:
ResultPhase2_2(d=d_9_city, n_node=10, w=w1_9, n_path=2,c=4,cr=0.4,n_pop=60,generation=150,min_dis_factor=1,min_dif_factor=1)

In [ ]:
# ResulฺtBruteForcePhase2_1(d=d_9_city, n_node=10, w=w1_9, n_path=2, c=4)

In [ ]:
# ResulฺtBruteForcePhase2_2(d=d_9_city, n_node=10, w=w1_9, n_path=2, c=4)

In [183]:
ResulฺtBruteForcePhase2_3(d=d_9_city, n_node=10, w=w1_9, n_path=2, c=4)

Brute Force
[0, 0, 0.8, 0, 0.9, 1.3, 1.5, 0, 0, 0.3]
[2, 4, 5, 6, 9]
Time used: 0 seconds
The shortest path is:  [[0, 6, 9, 5, 2, 4]]
Distance:  306
Different distance of two truck 306


In [ ]:
ResulฺtBruteForcePhase2_3(d=d_9_city, n_node=10, w=w2_9, n_path=2, c=4)

In [ ]:
ResulฺtBruteForcePhase2_4(d=d_9_city, n_node=10, w=w1_9, n_path=2, c=4)

In [ ]:
# ResultPhase2(d=d_9_city, n_node=10, w=w2_9, n_path=2,c=4)

In [ ]:
# ResulฺtBruteForcePhase2(d=d_9_city, n_node=10, w=w2_9, n_path=2, c=4)

#### 10 city

In [55]:
ResultPhase2_1(d=d_10_city, n_node=11, w=w1_10, n_path=2,c=4,cr=0.3,n_pop=50,generation=100)

Genetic Algorithm
Round  1
Time used: 0 seconds
The shortest path is:  [[0, 10, 4, 2, 5, 9], [0, 6]]
Distance:  466
Distance of truck [0, 10, 4, 2, 5, 9] 434
Distance of truck [0, 6] 32
Different distance of two truck 402
----------------------------------------------
Round  2
Time used: 0 seconds
The shortest path is:  [[0, 10, 4, 2, 5, 9], [0, 6]]
Distance:  466
Distance of truck [0, 10, 4, 2, 5, 9] 434
Distance of truck [0, 6] 32
Different distance of two truck 402
----------------------------------------------
Round  3
Time used: 0 seconds
The shortest path is:  [[0, 10, 4, 2, 5, 9], [0, 6]]
Distance:  466
Distance of truck [0, 10, 4, 2, 5, 9] 434
Distance of truck [0, 6] 32
Different distance of two truck 402
----------------------------------------------
Round  4
Time used: 0 seconds
The shortest path is:  [[0, 10, 4, 2, 5, 9], [0, 6]]
Distance:  466
Distance of truck [0, 10, 4, 2, 5, 9] 434
Distance of truck [0, 6] 32
Different distance of two truck 402
-------------------------

In [ ]:
ResultPhase2_2(d=d_10_city, n_node=11, w=w1_10, n_path=2,c=4,cr=0.4,n_pop=50,generation=100)

In [201]:
ResultPhase2_2(d=d_10_city, n_node=11, w=w1_10, n_path=2,c=4,cr=0.4,n_pop=60,generation=150,min_dis_factor=1,min_dif_factor=1)

Genetic Algorithm
Round  1
Initial population
Finish population
Start generation
Finish generation
Time used: 1 seconds
The shortest path is:  [[0, 4, 2, 5, 9, 6], [0, 10]]
Distance:  476
Distance of truck [0, 4, 2, 5, 9, 6] 306
Distance of truck [0, 10] 170
Different distance of two truck 136
----------------------------------------------
Round  2
Initial population
Finish population
Start generation
Finish generation
Time used: 1 seconds
The shortest path is:  [[0, 6, 5, 9, 2, 4], [0, 10]]
Distance:  476
Distance of truck [0, 6, 5, 9, 2, 4] 306
Distance of truck [0, 10] 170
Different distance of two truck 136
----------------------------------------------
Round  3
Initial population
Finish population
Start generation
Finish generation
Time used: 1 seconds
The shortest path is:  [[0, 6, 2, 4, 5, 9], [0, 10]]
Distance:  520
Distance of truck [0, 6, 2, 4, 5, 9] 350
Distance of truck [0, 10] 170
Different distance of two truck 180
----------------------------------------------
Round  4
I

In [56]:
ResultPhase2_2(d=d_10_city, n_node=11, w=w2_10, n_path=2,c=6,cr=0.7,n_pop=100,generation=300,min_dis_factor=1,min_dif_factor=1)

NameError: name 'ResultPhase2_2' is not defined

In [203]:
ResultPhase2_2(d=d_10_city, n_node=11, w=w2_10, n_path=2,c=6,cr=0.7,n_pop=100,generation=300,min_dis_factor=1,min_dif_factor=0)

Genetic Algorithm
Round  1
Initial population
Finish population
Start generation
Finish generation
Time used: 3 seconds
The shortest path is:  [[0, 3, 8, 9, 6, 1, 2, 5, 10], [0, 4, 7]]
Distance:  770
Distance of truck [0, 3, 8, 9, 6, 1, 2, 5, 10] 607
Distance of truck [0, 4, 7] 163
Different distance of two truck 444
----------------------------------------------
Round  2
Initial population
Finish population
Start generation
Finish generation
Time used: 3 seconds
The shortest path is:  [[0, 1, 8, 3, 6, 2, 9, 5, 10], [0, 4, 7]]
Distance:  737
Distance of truck [0, 1, 8, 3, 6, 2, 9, 5, 10] 574
Distance of truck [0, 4, 7] 163
Different distance of two truck 411
----------------------------------------------
Round  3
Initial population
Finish population
Start generation
Finish generation
Time used: 3 seconds
The shortest path is:  [[0, 5, 3, 8, 6, 9, 2, 4, 7], [0, 10, 1]]
Distance:  764
Distance of truck [0, 5, 3, 8, 6, 9, 2, 4, 7] 479
Distance of truck [0, 10, 1] 285
Different distance of

In [ ]:
# ResulฺtBruteForcePhase2_1(d=d_10_city, n_node=11, w=w1_10, n_path=2, c=4)

In [ ]:
# ResulฺtBruteForcePhase2_2(d=d_10_city, n_node=11, w=w1_10, n_path=2, c=4)

In [180]:
ResulฺtBruteForcePhase2_3(d=d_10_city, n_node=11, w=w1_10, n_path=2, c=4)

Brute Force
[0, 0, 0.8, 0, 0.9, 1.3, 1.5, 0, 0, 0.3, 0.2]
[2, 4, 5, 6, 9, 10]
Time used: 0 seconds
The shortest path is:  [[0, 6, 5, 9, 2, 4], [0, 10]]
Distance:  476
Different distance of two truck 136


In [184]:
ResulฺtBruteForcePhase2_3(d=d_10_city, n_node=11, w=w2_10, n_path=2, c=6)

Brute Force
[0, 0.2, 0.8, 0.3, 0.7, 0.5, 1, 1, 1, 2, 0.1]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Time used: 25 seconds
The shortest path is:  [[0, 3, 8, 1, 6, 5, 9, 2, 7], [0, 4, 10]]
Distance:  678
Different distance of two truck 34


In [ ]:
ResulฺtBruteForcePhase2_4(d=d_10_city, n_node=11, w=w1_10, n_path=2, c=4)

In [ ]:
# ResultPhase2(d=d_10_city, n_node=11, w=w2_10, n_path=2,c=5)

In [ ]:
# ResulฺtBruteForcePhase2(d=d_10_city, n_node=11, w=w2_10, n_path=2, c=5)

#### 11 city

In [ ]:
ResultPhase2_1(d=d_11_city, n_node=12, w=w1_11, n_path=2,c=4,cr=0.3,n_pop=50,generation=100)

In [ ]:
ResultPhase2_2(d=d_11_city, n_node=12, w=w1_11, n_path=2,c=4,cr=0.3,n_pop=50,generation=100)

In [ ]:
ResultPhase2_2(d=d_11_city, n_node=12, w=w1_11, n_path=2,c=4,cr=0.4,n_pop=60,generation=150,min_dis_factor=1,min_dif_factor=1)

In [ ]:
# ResulฺtBruteForcePhase2_1(d=d_11_city, n_node=12, w=w1_11, n_path=2, c=4)

In [ ]:
# ResulฺtBruteForcePhase2_2(d=d_11_city, n_node=12, w=w1_11, n_path=2, c=4)

In [ ]:
ResulฺtBruteForcePhase2_3(d=d_11_city, n_node=12, w=w1_11, n_path=2, c=4)

In [ ]:
ResulฺtBruteForcePhase2_3(d=d_11_city, n_node=12, w=w2_11, n_path=2, c=4)

In [ ]:
ResulฺtBruteForcePhase2_4(d=d_11_city, n_node=12, w=w1_11, n_path=2, c=4)

In [ ]:
# ResultPhase2(d=d_11_city, n_node=12, w=w2_11, n_path=2,c=5)

In [ ]:
# ResulฺtBruteForcePhase2(d=d_11_city, n_node=12, w=w2_11, n_path=2, c=5)

#### 12 city

In [ ]:
#ResultPhase2_1(d=d_12_city, n_node=13, w=w1_12, n_path=2,c=4,cr=0.3,n_pop=50,generation=100)

In [ ]:
ResultPhase2_2(d=d_12_city, n_node=13, w=w1_12, n_path=2,c=4,cr=0.3,n_pop=50,generation=100)

In [ ]:
#ResulฺtBruteForcePhase2_1(d=d_12_city, n_node=13, w=w1_12, n_path=2, c=4)

In [ ]:
#ResulฺtBruteForcePhase2_2(d=d_12_city, n_node=13, w=w1_12, n_path=2, c=4)

In [ ]:
ResulฺtBruteForcePhase2_3(d=d_12_city, n_node=13, w=w1_12, n_path=2, c=4)

In [ ]:
ResulฺtBruteForcePhase2_3(d=d_12_city, n_node=13, w=w2_12, n_path=2, c=4)

In [ ]:
# ResultPhase2(d=d_12_city, n_node=13, w=w2_12, n_path=2,c=5)

In [ ]:
# ResulฺtBruteForcePhase2(d=d_12_city, n_node=13, w=w2_12, n_path=2, c=5)

#### Big data set, with first demand constraint

In [ ]:
# Big data set, with demand constraint
x, y = genetic_algorithm(d=d_14_city, n_node=n_node_big, w=w1_14, n_path=n_path, c=c_big)
print("The shortest path is: ", x[-1])
print("Distance: ", y[-1]) 

In [ ]:
brute_force(d=d_big, n_node=n_node_big, w=w1, n_path=n_path, c=c_big)

In [ ]:
print("============ First Path =============")
d1 = sum_distance(d_14_city,x[-1][0],True)
print("============ Second Path =============")
d2 = sum_distance(d_14_city,x[-1][1],True)
print("======================================")
print("Distance: %.02lf" % (d1+d2))
print("======================================")

In [ ]:
print("============ First Path =============")
d1 = sum_demand(w1,x[-1][0],True)
print("============ Second Path =============")
d2 = sum_demand(w1,x[-1][1],True)
print("======================================")
print("All demand: ", sum_demand(w1))
print("Demand: %.02lf" % (d1+d2))
print("======================================")

#### Big data set with second demand constraint

In [ ]:
# Big data set, with another demand constraint
x, y = genetic_algorithm(d=d_14_city, n_node=15, w=w2_14, n_path=2, c=5)
print("The shortest path is: ", x[-1])
print("Distance: ", y[-1]) 

In [ ]:
print("============ First Path =============")
d1 = sum_distance(d_14_city,x[-1][0],True)
print("============ Second Path =============")
d2 = sum_distance(d_14_city,x[-1][1],True)
print("======================================")
print("Distance: %.02lf" % (d1+d2))
print("======================================")

In [ ]:
print("============ First Path =============")
d1 = sum_demand(w2_14,x[-1][0],True)
print("============ Second Path =============")
d2 = sum_demand(w2_14,x[-1][1],True)
print("======================================")
print("All demand: ", sum_demand(w2_14))
print("Demand: %.02lf" % (d1+d2))
print("======================================")